In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from matplotlib import rcParams
from collections import defaultdict
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential

## Importing Data

In [ ]:
tr= pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
pr =pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')

In [ ]:
tr.head()

In [ ]:
tr.info()

In [ ]:
pr.info()

## Analysis of Data

#### Finding Unique methods of crimes

In [ ]:
tr.groupby("Category")["Category"].count().sort_values(ascending=False)

In [ ]:
number_of_crimes = tr.Category.value_counts()
_n_crime_plot = sns.barplot(x=number_of_crimes.index,y=number_of_crimes)
_n_crime_plot.set_xticklabels(number_of_crimes.index,rotation=90)

#### Most Dangerous Day of week

In [ ]:
day_of_crimes = tr.DayOfWeek.value_counts()

_n_crime_plot = sns.scatterplot(x=day_of_crimes.index,y=day_of_crimes, color='red')
_n_crime_plot.set_xticklabels(day_of_crimes.index,rotation=90)

_n_crime_plot = sns.lineplot(x=day_of_crimes.index,y=day_of_crimes)
_n_crime_plot.set_xticklabels(day_of_crimes.index,rotation=90)

Apparently, Friday is the most sought-after day by criminals

#### Crime by district

In [ ]:
district = tr.PdDistrict.value_counts()

_n_crime_plot = sns.barplot(x=district.index,y=district)
_n_crime_plot.set_xticklabels(district.index,rotation=90)

Southern district is the most dangerous place

#### Occurence of crimes per month over the years

In [ ]:
tr['Dates'] = tr['Dates'].astype('datetime64[ns]')
tr['Month']= tr.Dates.dt.month
tr['Year']= tr.Dates.dt.year


pylab.rcParams['figure.figsize'] = (16.0, 5.0)
yearMonth = tr.groupby(['Year','Month']).size()
ax = yearMonth.plot(lw=2)
plt.title('San Francisco Crimes Monthly Over The Years', fontsize=26)
plt.show()

### ML Modelling

#### Getting dummies

In [ ]:
dow_dummy = pd.get_dummies(tr.DayOfWeek)
tr = pd.concat([tr, dow_dummy], axis=1)


dow_dummies = pd.get_dummies(pr.DayOfWeek)
pr = pd.concat([pr, dow_dummies], axis=1)


pd_dummy = pd.get_dummies(tr.PdDistrict)
tr = pd.concat([tr, pd_dummy], axis=1)


pd_dummies = pd.get_dummies(pr.PdDistrict)
pr = pd.concat([pr, pd_dummies], axis=1)


In [ ]:
tr.head()

In [ ]:
pr.drop('Address', axis=1, inplace=True)
pr.head()

In [ ]:
X=tr.drop(['Category'],axis=1)
y=pd.get_dummies(tr['Category'])

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1)

#### Preparartion of model (Sequential)

In [ ]:
MLmodel=Sequential()

MLmodel.add(Dense(256,input_shape=(X.shape[1],)))
MLmodel.add(Activation('relu'))

MLmodel.add(Dense(128))
MLmodel.add(Activation('relu'))

MLmodel.add(Dense(64))
MLmodel.add(Activation('relu'))

MLmodel.add(Dense(39))
MLmodel.add(Activation('softmax'))

In [ ]:
MLmodel.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
tr['Dates'] = pd.to_datetime(tr['Dates']).dt.time
pr['Dates'] = pd.to_datetime(pr['Dates']).dt.time

In [ ]:
tr['Dates'] = tr['Dates'].astype(str)
pr['Dates'] = pr['Dates'].astype(str)

In [ ]:
tr['Dates'] = tr['Dates'].replace(':','', regex=True)
pr['Dates'] = pr['Dates'].replace(':','', regex=True)

In [ ]:
tr['Dates'] = tr['Dates'].astype(int)
pr['Dates'] = pr['Dates'].astype(int)

In [ ]:
MLmodel.fit(X_train,y_train,
         batch_size=256,
         epochs=10,
         verbose=2,
         validation_data=(X_test,y_test))